<a href="https://colab.research.google.com/github/Cumberkid/Learning-the-Optimal-Solution-Path/blob/main/experiments/fair-regression/notebooks/07%20Learn%20Solution%20Path%20(LSP)%3A%20Legendre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary libraries

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader  #for creating the dataset


device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [ ]:
import pandas as pd

In [ ]:
# import importlib

## Import our own modules

In [ ]:
# !rm -r Learning-the-Optimal-Solution-Path
!git clone https://github.com/Cumberkid/Learning-the-Optimal-Solution-Path.git

Cloning into 'Learning-the-Optimal-Solution-Path'...
remote: Enumerating objects: 542, done.
remote: Counting objects: 100% (352/352), done.
remote: Compressing objects: 100% (239/239), done.
remote: Total 542 (delta 199), reused 180 (delta 105), pack-reused 190
Receiving objects: 100% (542/542), 10.99 MiB | 24.96 MiB/s, done.
Resolving deltas: 100% (295/295), done.


(Using Colab)

In [ ]:
import sys

In [ ]:
# Add the parent directory to sys.path
sys.path.append('/content/Learning-the-Optimal-Solution-Path')

In [ ]:
import lib
# importlib.reload(lib)

In [ ]:
from lib.utils_data import Regression_Data
from lib.lsp.basis_generator import phi_lam_legendre
from lib.lsp.basis_tf_module import Basis_TF_SGD
from lib.lsp.learn_solution_path import learn_solution_path

#Preliminaries
Recall that our method runs SGD over random $\tilde λ$'s with a linear basis $\Phi(\tilde \lambda)$ of our choice. We want to approximate $\theta$ with $\Phi(\lambda)\beta$, so the objective function is $\min_\beta h(\Phi(\tilde\lambda)\beta, \tilde\lambda) = (1-\tilde\lambda) BCE(X_\text{pass}\Phi(\tilde\lambda)\beta,\ y_\text{pass}) + \tilde\lambda BCE(X_\text{fail}\Phi(\tilde\lambda)\beta,\ y_\text{fail})$. For each batch of training data set, we randomize $\tilde\lambda$. If batch size = 1, then this is equivalent to a standard SGD.

## Load data

In [ ]:
# file path for Colab. May need to change this
X_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/X_processed.csv')
y_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/y_processed.csv')

In [ ]:
X = np.array(X_df)
y = np.array(y_df).squeeze()

In [ ]:
full_data = Regression_Data(X, y)
# full gradient descent uses all data points
GD_data_loader = DataLoader(full_data, batch_size=len(full_data), shuffle=True, )
# stochastic gradient descent uses mini-batch
SGD_data_loader = DataLoader(full_data, batch_size=20, shuffle=True, )
# test data
test_data_loader = DataLoader(full_data, batch_size=len(full_data), shuffle=False, )

In [ ]:
lam_max = 1
lam_min = 0
input_dim = X.shape[1]
criterion=torch.nn.BCELoss()

In [ ]:
# Read the CSV file into a DataFrame
truth = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/notebooks/results/exact_soln_list.csv')

# Display the DataFrame
truth

,losses,theta_0,theta_1,theta_2,theta_3,theta_4,theta_5,theta_6,theta_7,theta_8,...,theta_36,theta_37,theta_38,theta_39,theta_40,theta_41,theta_42,theta_43,theta_44,theta_45
0,0.000033,-2.734417,0.965609,1.547809,-0.461127,0.750339,0.785537,0.903840,0.609724,0.245244,...,-0.092991,0.353215,-0.022834,0.000000,0.000000,-0.237390,-0.092991,-0.696193,-0.774274,-0.910738
1,0.003984,-1.237352,0.868619,2.217027,-1.023908,0.577028,0.055938,-0.289084,-0.193347,0.491067,...,0.791513,-0.655901,-0.348404,0.159421,0.240847,-0.197884,0.791513,-1.152830,-0.358285,-0.382138
2,0.006419,-0.731118,0.968288,2.944446,-1.317535,0.325186,-0.051009,-0.400514,-1.247546,0.722822,...,1.094857,-0.983527,-0.912177,0.423760,0.587731,-0.237524,1.094857,-1.540321,0.028430,-0.202753
3,0.008501,-0.387372,1.302629,3.495369,-1.586220,0.168759,0.024970,-0.545149,-1.983109,0.889881,...,1.014987,-1.043256,-1.448549,0.780646,0.950633,-0.306635,1.014987,-1.490043,0.220487,-0.161065
4,0.010391,-0.077976,1.779688,3.941692,-1.796844,0.077043,0.097947,-0.660573,-2.422422,0.961997,...,0.867634,-1.124592,-1.918373,1.172459,1.296186,-0.378062,0.867634,-1.346151,0.244212,-0.100623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,0.017910,7.075102,1.749114,-2.494762,0.430425,1.145204,0.126876,-0.533614,-1.153567,0.292879,...,-2.301923,-5.683259,-3.637044,4.570588,4.259853,-2.773711,-2.301923,0.088017,0.576384,0.638555
1020,0.014202,7.312490,1.760152,-2.559923,0.448954,1.160237,0.125508,-0.543709,-1.173318,0.295185,...,-2.281472,-5.710746,-3.642248,4.570588,4.259853,-2.761513,-2.281472,0.162650,0.660690,0.689425
1021,0.010200,7.626506,1.760365,-2.625994,0.467065,1.169712,0.119306,-0.551297,-1.194783,0.299307,...,-2.252983,-5.749014,-3.646690,4.570588,4.259853,-2.747193,-2.252983,0.259606,0.771384,0.757299
1022,0.005740,8.071383,1.735432,-2.705320,0.463105,1.163599,0.093488,-0.552707,-1.209416,0.308961,...,-2.206589,-5.814081,-3.646463,4.570588,4.259853,-2.728747,-2.206589,0.390584,0.925827,0.851714


In [ ]:
selected_columns = ['theta_0', 'theta_1', 'theta_2', 'theta_3', 'theta_4',
                    'theta_5', 'theta_6', 'theta_7', 'theta_8', 'theta_9',
                    'theta_10', 'theta_11', 'theta_12', 'theta_13', 'theta_14',
                    'theta_15', 'theta_16', 'theta_17', 'theta_18', 'theta_19',
                    'theta_20', 'theta_21', 'theta_22', 'theta_23', 'theta_24',
                    'theta_25', 'theta_26', 'theta_27', 'theta_28', 'theta_29',
                    'theta_30', 'theta_31', 'theta_32', 'theta_33', 'theta_34',
                    'theta_35', 'theta_36', 'theta_37', 'theta_38', 'theta_39',
                    'theta_40', 'theta_41', 'theta_42', 'theta_43', 'theta_44',
                    'theta_45']
true_thetas = truth[selected_columns].to_numpy()
true_losses = truth['losses'].to_numpy()

## Choose basis functions

We use Legendre polynomials with degree $\leq n$ as the basis vectors for $\Phi(\lambda)$.

In [ ]:
phi_lam = phi_lam_legendre

In [ ]:
criterion = torch.nn.BCELoss()
input_dim = X.shape[1]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time

# Exact Gradient Oracle Diminishing LR

We use diminishing learning rate for better demonstrate convergence. If we use a constant learning rate, the solution path error will eventually do a random walk after descending to a certain threshold value.

We will see this random walk in a plot later.

In [ ]:
epochs = 20000

## num basis func = 3

In [ ]:
basis_dim = 3
lr = 0.5

np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, model = learn_solution_path(input_dim, basis_dim, phi_lam, epochs,
                                                                          GD_data_loader, test_data_loader, criterion,
                                                                          lam_min, lam_max, true_losses,
                                                                          lr=lr, diminish=True, gamma=0.97, dim_step=100,
                                                                          obj='fairness',
                                                                          record_frequency=50, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# itr: 100	 sup error: 0.10143660148241906
--------approximate solution path for # itr = 200 complete--------
# itr: 200	 sup error: 0.05137653648853305
--------approximate solution path for # itr = 300 complete--------
# itr: 300	 sup error: 0.04835073649883273
--------approximate solution path for # itr = 400 complete--------
# itr: 400	 sup error: 0.04287533973911195
--------approximate solution path for # itr = 500 complete--------
# itr: 500	 sup error: 0.07698930939295678
--------approximate solution path for # itr = 600 complete--------
# itr: 600	 sup error: 0.05096540557133267
--------approximate solution path for # itr = 700 complete--------
# itr: 700	 sup error: 0.08986637880661874
--------approximate solution path for # itr = 800 complete--------
# itr: 800	 sup error: 0.03342323334800312
--------approximate solution path for # itr = 900 complete--------
# itr: 900	 sup error: 0.06837133457520395
--------ap

In [ ]:
file_path = 'SGD_results_exact_diminish.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df['sup_err_3'] = sup_err_history

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

## num basis func = 5

In [ ]:
basis_dim = 5
lr = 0.5

np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, model = learn_solution_path(input_dim, basis_dim, phi_lam, epochs,
                                                                          GD_data_loader, test_data_loader, criterion,
                                                                          lam_min, lam_max, true_losses,
                                                                          lr=lr, diminish=True, gamma=0.97, dim_step=100,
                                                                          obj='fairness',
                                                                          record_frequency=50, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# itr: 100	 sup error: 0.0615294873714447
--------approximate solution path for # itr = 200 complete--------
# itr: 200	 sup error: 0.04150190949440008
--------approximate solution path for # itr = 300 complete--------
# itr: 300	 sup error: 0.03993334387996583
--------approximate solution path for # itr = 400 complete--------
# itr: 400	 sup error: 0.033853560686111506
--------approximate solution path for # itr = 500 complete--------
# itr: 500	 sup error: 0.046571688257245114
--------approximate solution path for # itr = 600 complete--------
# itr: 600	 sup error: 0.028278935271373484
--------approximate solution path for # itr = 700 complete--------
# itr: 700	 sup error: 0.02525292336940771
--------approximate solution path for # itr = 800 complete--------
# itr: 800	 sup error: 0.03326881443717866
--------approximate solution path for # itr = 900 complete--------
# itr: 900	 sup error: 0.023396581411361778
-------

In [ ]:
# file_path = '/content/drive/MyDrive/Experiments/SGD_results_exact.csv'
file_path = 'SGD_results_exact_diminish.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df['sup_err_5'] = sup_err_history

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

## num basis func = 7

In [ ]:
basis_dim = 7
lr = 0.5

np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, model = learn_solution_path(input_dim, basis_dim, phi_lam, epochs,
                                                                          GD_data_loader, test_data_loader, criterion,
                                                                          lam_min, lam_max, true_losses,
                                                                          lr=lr, diminish=True, gamma=0.97, dim_step=100,
                                                                          obj='fairness',
                                                                          record_frequency=50, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# itr: 100	 sup error: 0.1305037974343577
--------approximate solution path for # itr = 200 complete--------
# itr: 200	 sup error: 1.1327693699822703
--------approximate solution path for # itr = 300 complete--------
# itr: 300	 sup error: 0.03901168704032898
--------approximate solution path for # itr = 400 complete--------
# itr: 400	 sup error: 0.034261688590049744
--------approximate solution path for # itr = 500 complete--------
# itr: 500	 sup error: 0.0312637984752655
--------approximate solution path for # itr = 600 complete--------
# itr: 600	 sup error: 0.028612881898880033
--------approximate solution path for # itr = 700 complete--------
# itr: 700	 sup error: 0.026007816195487976
--------approximate solution path for # itr = 800 complete--------
# itr: 800	 sup error: 0.02480718493461609
--------approximate solution path for # itr = 900 complete--------
# itr: 900	 sup error: 0.023870542645454407
--------a

In [ ]:
file_path = 'SGD_results_exact_diminish.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df['sup_err_7'] = sup_err_history

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

## num basis func = 9

In [ ]:
basis_dim = 9
lr = 0.4

np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, model = learn_solution_path(input_dim, basis_dim, phi_lam, epochs,
                                                                          GD_data_loader, test_data_loader, criterion,
                                                                          lam_min, lam_max, true_losses,
                                                                          lr=lr, diminish=False, gamma=0.99, dim_step=100,
                                                                          obj='fairness',
                                                                          record_frequency=50, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# itr: 100	 sup error: 0.232819139957428
--------approximate solution path for # itr = 200 complete--------
# itr: 200	 sup error: 0.10918211936950689
--------approximate solution path for # itr = 300 complete--------
# itr: 300	 sup error: 0.10591945052146912
--------approximate solution path for # itr = 400 complete--------
# itr: 400	 sup error: 0.07886156579115777
--------approximate solution path for # itr = 500 complete--------
# itr: 500	 sup error: 0.03160020709037781
--------approximate solution path for # itr = 600 complete--------
# itr: 600	 sup error: 0.07899412512779241
--------approximate solution path for # itr = 700 complete--------
# itr: 700	 sup error: 0.14214714946137974
--------approximate solution path for # itr = 800 complete--------
# itr: 800	 sup error: 0.038662489074340556
--------approximate solution path for # itr = 900 complete--------
# itr: 900	 sup error: 0.054818198084831266
--------ap

In [ ]:
# file_path = '/content/drive/MyDrive/Experiments/SGD_results_exact.csv'
file_path = 'SGD_results_exact_diminish.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df['sup_err_9'] = sup_err_history

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

# Exact Gradient Oracle Constant LR

Train the SGD model for our method using exact gradient and record the sup error along the solution path ($\epsilon$) achieved after executing some number of gradient calls (epochs).

We use the previously tuned learning rate $0.5$.

In [ ]:
epochs = 200000

## num basis func = 3

In [ ]:
basis_dim = 3
lr = 0.5

np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, model = learn_solution_path(input_dim, basis_dim, phi_lam, epochs,
                                                                   GD_data_loader, test_data_loader, criterion,
                                                                   lam_min, lam_max, true_losses,
                                                                   lr=lr, SGD=False, obj='fairness',
                                                                   intercept=True, trace_frequency=1000)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 1000 complete--------
# itr: 1000	 sup error: 0.0391506305859366
--------approximate solution path for # itr = 2000 complete--------
# itr: 2000	 sup error: 0.0689924839316518
--------approximate solution path for # itr = 3000 complete--------
# itr: 3000	 sup error: 0.058994860850361874
--------approximate solution path for # itr = 4000 complete--------
# itr: 4000	 sup error: 0.06420439621069818
--------approximate solution path for # itr = 5000 complete--------
# itr: 5000	 sup error: 0.07453113188603311
--------approximate solution path for # itr = 6000 complete--------
# itr: 6000	 sup error: 0.07240858813383966
--------approximate solution path for # itr = 7000 complete--------
# itr: 7000	 sup error: 0.03846745704868226
--------approximate solution path for # itr = 8000 complete--------
# itr: 8000	 sup error: 0.06291342666008859
--------approximate solution path for # itr = 9000 complete--------
# itr: 9000	 sup error: 0.03311571063

In [ ]:
# import csv

# csv_file_path = '/content/drive/MyDrive/Experiments/SGD_results_exact.csv'

# # Open the CSV file for writing
# with open(csv_file_path, 'w', newline='') as csv_file:
#     # Create a CSV writer
#     csv_writer = csv.writer(csv_file)

#     # Write the headers
#     csv_writer.writerow(['num_itr', 'sup_err_13'])

#     # Write the data from your lists
#     for i in range(len(epochs)):
#         csv_writer.writerow([epochs[i], sup_err_history[i]])

In [ ]:
# for i in range(len(sup_err_history)):
#     sup_err_history[i] = np.min(sup_err_history[:i+1])

In [ ]:
# file_path = '/content/drive/MyDrive/Experiments/SGD_results_exact.csv'
file_path = 'SGD_results_exact.csv'

SGD_results_exact = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err_3'])

# Save the DataFrame to a CSV file
SGD_results_exact.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

## num basis func = 5

In [ ]:
basis_dim = 5
lr = 0.5

np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, model = learn_solution_path(input_dim, basis_dim, phi_lam, epochs,
                                                                   GD_data_loader, test_data_loader, criterion,
                                                                   lam_min, lam_max, true_losses,
                                                                   lr=lr, SGD=False, obj='fairness',
                                                                   intercept=True, trace_frequency=1000)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 1000 complete--------
# itr: 1000	 sup error: 0.021212071180343656
--------approximate solution path for # itr = 2000 complete--------
# itr: 2000	 sup error: 0.027864938132552197
--------approximate solution path for # itr = 3000 complete--------
# itr: 3000	 sup error: 0.03125660260775476
--------approximate solution path for # itr = 4000 complete--------
# itr: 4000	 sup error: 0.025707814373163274
--------approximate solution path for # itr = 5000 complete--------
# itr: 5000	 sup error: 0.020930639930156758
--------approximate solution path for # itr = 6000 complete--------
# itr: 6000	 sup error: 0.025872727386740735
--------approximate solution path for # itr = 7000 complete--------
# itr: 7000	 sup error: 0.01734619358467171
--------approximate solution path for # itr = 8000 complete--------
# itr: 8000	 sup error: 0.026055395603180015
--------approximate solution path for # itr = 9000 complete--------
# itr: 9000	 sup error: 0.0266

In [ ]:
# Add a new column for a different basis dimension
df['sup_err_5'] = sup_err_history

# Save the updated DataFrame back to the CSV file
df.to_csv(file_path, index=False)

## num basis func = 7

In [ ]:
epochs = 530000

basis_dim = 7
lr = 0.5

num_itr_history, sup_err_history = SGD.learn_solution_path(input_dim, basis_dim, phi_lam, epochs,
                                                                   GD_data_loader, test_data_loader, criterion,
                                                                   lam_min, lam_max, true_losses,
                                                                   lr=lr, SGD=False, obj='fairness',
                                                                   intercept=True, trace_frequency=1000)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 1000 complete--------
# itr: 1000	 sup error: 0.02379039628067403
--------approximate solution path for # itr = 2000 complete--------
# itr: 2000	 sup error: 0.01474568247795105
--------approximate solution path for # itr = 3000 complete--------
# itr: 3000	 sup error: 0.011237071357754758
--------approximate solution path for # itr = 4000 complete--------
# itr: 4000	 sup error: 0.02960283448555856
--------approximate solution path for # itr = 5000 complete--------
# itr: 5000	 sup error: 0.010425486172607634
--------approximate solution path for # itr = 6000 complete--------
# itr: 6000	 sup error: 0.01814755005398183
--------approximate solution path for # itr = 7000 complete--------
# itr: 7000	 sup error: 0.04265592386582284
--------approximate solution path for # itr = 8000 complete--------
# itr: 8000	 sup error: 0.01884514605262666
--------approximate solution path for # itr = 9000 complete--------
# itr: 9000	 sup error: 0.01223029

In [ ]:
# Add a new column for a different basis dimension
df['sup_err_7'] = sup_err_history[:200000]

# Save the updated DataFrame back to the CSV file
df.to_csv(file_path, index=False)

## num basis func = 9

In [ ]:
basis_dim = 9
lr = 0.25

np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, model = learn_solution_path(input_dim, basis_dim, phi_lam, epochs,
                                                                   GD_data_loader, test_data_loader, criterion,
                                                                   lam_min, lam_max, true_losses,
                                                                   lr=lr, SGD=False, obj='fairness',
                                                                   intercept=True, trace_frequency=1000)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 1000 complete--------
# itr: 1000	 sup error: 0.029148578643798884
--------approximate solution path for # itr = 2000 complete--------
# itr: 2000	 sup error: 0.01856686174869543
--------approximate solution path for # itr = 3000 complete--------
# itr: 3000	 sup error: 0.015547968447208488
--------approximate solution path for # itr = 4000 complete--------
# itr: 4000	 sup error: 0.0133076682686806
--------approximate solution path for # itr = 5000 complete--------
# itr: 5000	 sup error: 0.01187230646610267
--------approximate solution path for # itr = 6000 complete--------
# itr: 6000	 sup error: 0.010651290416717599
--------approximate solution path for # itr = 7000 complete--------
# itr: 7000	 sup error: 0.01139620551475673
--------approximate solution path for # itr = 8000 complete--------
# itr: 8000	 sup error: 0.009030185639858301
--------approximate solution path for # itr = 9000 complete--------
# itr: 9000	 sup error: 0.0111460

In [ ]:
# file_path = '/content/drive/MyDrive/Experiments/SGD_results_exact.csv'
df = pd.read_csv(file_path)

# df = df.rename(columns={'sup_err': 'sup_err_9'})

# Add a new column for a different basis dimension
df['sup_err_9'] = sup_err_history

# Save the updated DataFrame back to the CSV file
df.to_csv(file_path, index=False)

# Noisy Gradient Oracle Diminishing LR

Train the SGD model for our method using noisy gradient (mini-batch SGD) and record the sup error along the solution path ($\epsilon$) achieved after executing some number of gradient calls (epochs).

We use the previously tuned shrinking rate $\alpha = 4$.

In [ ]:
epochs = 20000

## Num basis func = 3

In [ ]:
basis_dim = 3
alpha = 4
init_lr = 0.1

np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, model = learn_solution_path(input_dim, basis_dim, phi_lam, epochs,
                                                                          SGD_data_loader, test_data_loader, criterion,
                                                                          lam_min, lam_max, true_losses,
                                                                          alpha=alpha, init_lr=init_lr, SGD=True,
                                                                          obj='fairness',
                                                                          record_frequency=50, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# itr: 100	 sup error: 0.07442298861133168
--------approximate solution path for # itr = 200 complete--------
# itr: 200	 sup error: 0.05685558499681065
--------approximate solution path for # itr = 300 complete--------
# itr: 300	 sup error: 0.06379422875033924
--------approximate solution path for # itr = 400 complete--------
# itr: 400	 sup error: 0.050471558737626765
--------approximate solution path for # itr = 500 complete--------
# itr: 500	 sup error: 0.051018595266214106
--------approximate solution path for # itr = 600 complete--------
# itr: 600	 sup error: 0.04237372757779667
--------approximate solution path for # itr = 700 complete--------
# itr: 700	 sup error: 0.049319497811666224
--------approximate solution path for # itr = 800 complete--------
# itr: 800	 sup error: 0.04577848272310803
--------approximate solution path for # itr = 900 complete--------
# itr: 900	 sup error: 0.04536854432808468
-------

In [ ]:
# import csv

# csv_file_path = '/content/drive/MyDrive/Experiments/SGD_results_noisy.csv'

# # Open the CSV file for writing
# with open(csv_file_path, 'w', newline='') as csv_file:
#     # Create a CSV writer
#     csv_writer = csv.writer(csv_file)

#     # Write the headers
#     csv_writer.writerow(['num_itr', 'sup_err_3'])

#     # Write the data from your lists
#     for i in range(len(epochs)):
#         csv_writer.writerow([epochs[i], sup_err_history[i]])

In [ ]:
# for i in range(len(sup_err_history)):
#     sup_err_history[i] = np.min(sup_err_history[:i+1])
# # print(model_SGD.linear.weight)

In [ ]:
file_path = 'SGD_results_noisy.csv'

SGD_results_noisy = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err_3'])

# Save the DataFrame to a CSV file
SGD_results_noisy.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_itr,sup_err_3
0,50.0,0.079872
1,100.0,0.074423
2,150.0,0.055727
3,200.0,0.056856
4,250.0,0.061956
...,...,...
395,19800.0,0.047142
396,19850.0,0.047238
397,19900.0,0.046526
398,19950.0,0.046902


## Num basis func = 4

In [ ]:
basis_dim = 4
alpha = 4
init_lr = 0.1

np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, model = learn_solution_path(input_dim, basis_dim, phi_lam, epochs,
                                                                   SGD_data_loader, test_data_loader, criterion,
                                                                   lam_min, lam_max, true_losses,
                                                                   alpha=alpha, init_lr=init_lr, SGD=True,
                                                                   obj='fairness',
                                                                   record_frequency=50, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# itr: 100	 sup error: 0.06639673145400593
--------approximate solution path for # itr = 200 complete--------
# itr: 200	 sup error: 0.04096347093582159
--------approximate solution path for # itr = 300 complete--------
# itr: 300	 sup error: 0.02890646457672119
--------approximate solution path for # itr = 400 complete--------
# itr: 400	 sup error: 0.03077143430709839
--------approximate solution path for # itr = 500 complete--------
# itr: 500	 sup error: 0.030136495828628596
--------approximate solution path for # itr = 600 complete--------
# itr: 600	 sup error: 0.03331947326660156
--------approximate solution path for # itr = 700 complete--------
# itr: 700	 sup error: 0.028315693140029907
--------approximate solution path for # itr = 800 complete--------
# itr: 800	 sup error: 0.03309449553489685
--------approximate solution path for # itr = 900 complete--------
# itr: 900	 sup error: 0.02817434072494507
--------

In [ ]:
file_path = 'SGD_results_noisy.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Add a new column for basis dimension 5
df['sup_err_4'] = sup_err_history

# Save the updated DataFrame back to the CSV file
df.to_csv(file_path, index=False)

## num basis func = 5

In [ ]:
basis_dim = 5
alpha = 4
init_lr = 0.1

np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history = learn_solution_path(input_dim, basis_dim, phi_lam, epochs,
                                                                   SGD_data_loader, test_data_loader, criterion,
                                                                   lam_min, lam_max, true_losses,
                                                                   alpha=alpha, init_lr=init_lr, SGD=True,
                                                                   obj='fairness',
                                                                   record_frequency=50, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# itr: 100	 sup error: 0.04469746351242071
--------approximate solution path for # itr = 200 complete--------
# itr: 200	 sup error: 0.04195532202720642
--------approximate solution path for # itr = 300 complete--------
# itr: 300	 sup error: 0.041019111871719416
--------approximate solution path for # itr = 400 complete--------
# itr: 400	 sup error: 0.03226757049560547
--------approximate solution path for # itr = 500 complete--------
# itr: 500	 sup error: 0.02687874436378479
--------approximate solution path for # itr = 600 complete--------
# itr: 600	 sup error: 0.03545558452606201
--------approximate solution path for # itr = 700 complete--------
# itr: 700	 sup error: 0.0295568257570267
--------approximate solution path for # itr = 800 complete--------
# itr: 800	 sup error: 0.030685067176818903
--------approximate solution path for # itr = 900 complete--------
# itr: 900	 sup error: 0.02987194061279297
--------a

In [ ]:
file_path = 'SGD_results_noisy.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Add a new column for basis dimension 5
df['sup_err_5'] = sup_err_history

# Save the updated DataFrame back to the CSV file
df.to_csv(file_path, index=False)

## num basis func = 7

In [ ]:
basis_dim = 7
alpha = 4
init_lr = 0.1

np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, model = learn_solution_path(input_dim, basis_dim, phi_lam, epochs,
                                                                   SGD_data_loader, test_data_loader, criterion,
                                                                   lam_min, lam_max, true_losses,
                                                                   alpha=alpha, init_lr=init_lr, SGD=True,
                                                                   obj='fairness',
                                                                   record_frequency=50, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# itr: 100	 sup error: 0.05863964557647705
--------approximate solution path for # itr = 200 complete--------
# itr: 200	 sup error: 0.06315469741821289
--------approximate solution path for # itr = 300 complete--------
# itr: 300	 sup error: 0.04961672425270086
--------approximate solution path for # itr = 400 complete--------
# itr: 400	 sup error: 0.04473420977592468
--------approximate solution path for # itr = 500 complete--------
# itr: 500	 sup error: 0.03390395641326904
--------approximate solution path for # itr = 600 complete--------
# itr: 600	 sup error: 0.032155692577362116
--------approximate solution path for # itr = 700 complete--------
# itr: 700	 sup error: 0.033953577280044556
--------approximate solution path for # itr = 800 complete--------
# itr: 800	 sup error: 0.03640353679656988
--------approximate solution path for # itr = 900 complete--------
# itr: 900	 sup error: 0.027862071990966797
-------

In [ ]:
# file_path = '/content/drive/MyDrive/Experiments/SGD_results_noisy.csv'
df = pd.read_csv(file_path)

# Add a new column for a different basis dimension
df['sup_err_7'] = sup_err_history

# Save the updated DataFrame back to the CSV file
df.to_csv(file_path, index=False)

In [ ]:
# # plot result of learning the solution path
# plt.figure(figsize=(18,7))
# plt.plot(torch.log(torch.tensor(sup_err_history)), epochs, '--o')
# plt.ylabel('# of Gradient Calls')
# plt.xlabel('$\\log\\epsilon$')
# plt.legend()
# plt.grid(True)